In [ ]:
import pandas as pd

df = pd.DataFrame({"Name": ["blue", 
                            "delta", 
                            "echo", 
                            "charlie", 
                            "alpha"], 
                    "Type": ["Raptors",
                            "Raptors",
                            "Raptors",
                            "Raptors",
                            "Tyrannosaurus rex"]
                    })

print (df.index[(df['Name'] == 'blue')].tolist())
print ('\n')
print (df.loc[df['Name'] == 'blue'])
print ('\n')
print (df.loc[(df['Name'] == 'charlie') & (df['Type'] =='Raptors')])

In [ ]:
import mysql
import mysql.connector
conn = mysql.connector.connect(host='localhost', port='3306', database='mypictures', user='root', password='1234')
cursor = conn.cursor(dictionary=True)
cursor.execute('select pictureFileSrcDir from mypicturefiles;')
result = cursor.fetchall()
#print(result['pictureFileSrcDir'])
for dirs in result:
    print(dirs['pictureFileSrcDir'])
print()
cursor.execute('select pictureFileSrcDir from mypicturefiles;')
result = cursor.fetchone()
print(result['pictureFileSrcDir'])
              
#print(result)
#excute.

#existDF = pd.DataFrame(index=range(0, 0), columns={'dirName', 'fileName'})
#print(existDF)

In [ ]:
import os
import re
import pathlib
import stat
from datetime import datetime
from PIL import Image
from PIL.ExifTags import TAGS
from time import time
import time
import mysql
import mysql.connector

#path = 'E:\\source\\pygame\\Nado Game\\pyqt5\\사진관리\\data'
path = 'C:\\사진정리\\2004년\\2004년01월'
file = '20060116_132458_DSCF2972.JPG'
def listFiles():
    return os.walk(path)

def checkReMatch(path):
    checker = re.compile(r'(19\d\d|20\d\d|\d\d)[년\-_. ]?(0[1-9]|1[012])[월\-_. ]?(0[1-9]|[12][0-9]|3[01])[일]?')
    return checker.match(path)

def get_remark(path):
    lastDir = os.path.basename(path)
    m = checkReMatch(lastDir)
    if m and len(lastDir) > m.end():
        #print(sub[m.end():])
        return lastDir[m.end():]
    else:
        return ""

def connDB():
    conn = mysql.connector.connect(
        host='localhost', 
        port='3306', 
        database='mypictures', 
        user='root', 
        password='1234'
    )
    return conn
def takePictureTime(path, f):
    filename = pathlib.Path(path, f)
    try :
        with Image.open(filename) as im:
            info = im._getexif()
        # 새로운 딕셔너리 생성
        taglabel = {}
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            taglabel[decoded] = value
        s = taglabel['DateTimeOriginal']
        timestamp = time.mktime(datetime.strptime(s, '%Y:%m:%d %H:%M:%S').timetuple())
        return s
        #return timestamp
    except:
        return None

def fileToDB():
    files = listFiles()
    
    for path, subDir, files in files:
        for file in files:
            #print(file)
            remark = get_remark(path)
            tt = takePictureTime(path, file)
            exist = selectDB([str(file)])
            #print('exist: ', exist)
            if not exist:
                print('not exist insertDB: ', path, file)
                insertDB(file, path, path, tt, remark)
            else:
                dupliExist = selectDupliDB(file, path)
                #print(dupliExist)
                if not dupliExist:
                    print('not duplicated insertDupliDB: ', path, file)
                    insertDupliDB(file, path, path, remark)
                
def selectDupliDB(file, path):
    conn = connDB()
    #if conn.is_connected():
    #print(True)
    cursor = conn.cursor(dictionary=True)
    sql = "select * from duplicatedpicturefiles \
        where duPicFileName = %s and srcDir = %s;"
    val = (file, path)
    cursor.execute(sql, val)
    #print(cursor)
    result = cursor.fetchall()
    conn.close()
    return result


def insertDupliDB(f, srcDir, desDir, d):
    print(d)
    conn = connDB()
    cursor = conn.cursor(dictionary=True)
    sql = "insert into duplicatedpicturefiles (duPicFileName, srcDir, destDir, action) \
    values (%s, %s, %s, %s)"
    val = (f, srcDir, desDir, 'Delete')
    cursor.execute(sql, val)
    conn.commit()
    conn.close()
    return

def insertDB(f, destDir, srcDir, TT, remark):
    conn = connDB()
    cursor = conn.cursor(dictionary=True)
    sql = "insert into mypicturefiles (pictureFileName, pictureFileDestDir, pictureFileSrcDir, TakeTime, remark) \
    values (%s, %s, %s, %s, %s)"
    val = (f, destDir, srcDir, TT, remark)
    cursor.execute(sql, val)
    conn.commit()
    conn.close()
    return

def selectDB(f, path):
    conn = connDB()
    #if conn.is_connected():
    #print(True)
    cursor = conn.cursor(dictionary=True)
    sql = "select * from mypicturefiles \
    where pictureFileName = %s;"
    val = (f)
    cursor.execute(sql, val)
    #print(cursor)
    result = cursor.fetchall()
    conn.close()
    return result
    
    
fileToDB()    


In [ ]:
len('20230225_172255_027a91dfa667101abe36030cd10e81971eca8bdf2ac1f0e3b20a2449ba9c8a43.png')

In [ ]:
text= ('D:\사진\2011년\2011년10월\2011년10월01일 BNPP 1N2 Excavation ceremony') 
print(len(text))
len(text.encode())